In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("results/data/model-1-2025-03-15-17-27-07.csv")
df

In [ ]:
df = pd.read_csv("results/data/model-2-2025-03-15-17-27-15.csv")
df

In [ ]:
df = pd.read_csv("results/data/model-3-2025-03-15-17-33-10.csv")
df

In [4]:
import psutil
import cpuinfo


def estimate_cpu_peak_flops():
    # 1. Get CPU frequency (max) in MHz
    freq_info = psutil.cpu_freq()
    # freq_info.max is in MHz; convert to GHz
    cpu_freq_ghz = freq_info.max / 1000.0 if freq_info.max else freq_info.current / 1000.0

    # 2. Get the number of physical cores
    physical_cores = psutil.cpu_count(logical=False)

    # 3. Decide on a per-cycle floating point ops factor:
    #    For SSE (128-bit) with FP32 => 4 ops/cycle per core
    #    For AVX (256-bit) with FP32 => 8 ops/cycle per core
    #    For AVX-512 (512-bit) with FP32 => 16 ops/cycle per core
    #    If FMA is supported, we often double it because a fused multiply-add can handle 2 ops in one instruction.
    #
    #    This is a big simplification. You may look up your CPU's actual SIMD support.
    #    We'll guess AVX2 + FMA => 8 ops/cycle * 2 = 16 ops/cycle per core
    #    (again, approximate).

    # Check for some CPU flags
    info = cpuinfo.get_cpu_info()
    print(info)
    flags = info.get("flags", [])

    # Default to SSE
    simd_ops_per_cycle = 4
    simd_label = "SSE (128-bit)"
    print(f"Standard simd label: {simd_label}")
    print(flags)

    # If AVX2
    if "avx2" in flags:
        simd_ops_per_cycle = 8
        simd_label = "AVX2 (256-bit)"
        print(f"New simd_label: {simd_label}")

    # If AVX-512
    if "avx512f" in flags:
        simd_ops_per_cycle = 16
        simd_label = "AVX-512 (512-bit)"
        print(f"New simd_label: {simd_label}")

    # Decide if FMA is available
    # If we have FMA, we can roughly double the ops
    fma_factor = 2 if "fma" in flags else 1

    ops_per_cycle_per_core = simd_ops_per_cycle * fma_factor

    # 4. Compute approximate FLOPs
    # freq in GHz * 1e9 = cycles per second
    # then multiply by number of ops/cycle/core * cores
    peak_flops = cpu_freq_ghz * 1e9 * physical_cores * ops_per_cycle_per_core

    return peak_flops, cpu_freq_ghz, physical_cores, simd_label, bool("fma" in flags)


if __name__ == "__main__":
    peak, ghz, cores, simd_label, fma_found = estimate_cpu_peak_flops()
    print(f"CPU Frequency (GHz): {ghz:.2f}")
    print(f"Physical Cores:     {cores}")
    print(f"SIMD Detected:      {simd_label}")
    print(f"FMA Supported:      {fma_found}")
    print(f"Approx. Peak FLOPs: {peak:.2e} FLOP/s")

{'python_version': '3.11.2.final.0 (64 bit)', 'cpuinfo_version': [9, 0, 0], 'cpuinfo_version_string': '9.0.0', 'arch': 'ARM_8', 'bits': 64, 'count': 11, 'arch_string_raw': 'arm64', 'brand_raw': 'Apple M3 Pro'}
Standard simd label: SSE (128-bit)
[]
CPU Frequency (GHz): 4.06
Physical Cores:     11
SIMD Detected:      SSE (128-bit)
FMA Supported:      False
Approx. Peak FLOPs: 1.78e+11 FLOP/s
